In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

df = pd.read_csv('training_data.csv')
test = pd.read_csv('test_data.csv')
print(test.head(5))
print(df.head(5))
outid = test['review_id'].values # dataframe to numpy
#print(outid.T)
#print(len(outid))
train = df

   review_id  business_id  user_id  \
0       2713          668     2196   
1       4734         1014     3521   
2       5598         2939     3028   
3       9545         4077     6200   
4       1471         1126     1268   

                                                text        date  
0  Ticoz is not a bad place.  The menu is eclecti...  2009-01-04  
1  Wow!\r\nBeing a Phoenix native, we have enjoye...  2010-10-05  
2  Jodi is fantastic!  Extraordinary!  After too ...  2011-04-29  
3  T&S Glass did a wonderful job replacing my car...  2007-10-02  
4  My experience was on a random Saturday afterno...  2010-08-16  
   review_id  business_id  user_id  \
0       3223         2055     2533   
1       9938         4165     6371   
2       7123          869     4929   
3       3601         1603     2789   
4       3948         2347     1245   

                                                text        date  stars  
0  Sometimes things happen, and when they do this...  2010-12-30  

In [2]:
from sklearn.svm import LinearSVC

X_train = df.drop(['review_id','business_id','user_id','date'], axis=1) 

# use tfidf 
vectorizer = TfidfVectorizer()#stop_words="english",min_df = 0.01) 
vectorizer.fit(train['text'])

X_train = vectorizer.transform(train['text'])
X_test = vectorizer.transform(test['text'])
y_train = train['stars'].as_matrix()

# use the linearSVC model
clf = LinearSVC() # default = 1000
clf = clf.fit(X_train, y_train)
pred_y = clf.predict(X_test)

ans1 = pred_y
print(clf.score(X_train, y_train))

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()


0.9707390271351757


In [3]:
from sklearn.svm import LinearSVR

# set ngram the 1 to 3 combine words will be counted 
vectorizer = TfidfVectorizer(ngram_range=[1,3])#,max_df = 2000)#,min_df = 0.01)#,stop_words="english",min_df = 0.009) 
vectorizer.fit(train['text'])

X_train = vectorizer.transform(train['text'])
X_test = vectorizer.transform(test['text'])
y_train = train['stars'].as_matrix()

# use linearSVR model
clf2 = LinearSVR() # default = 1000
clf2 = clf2.fit(X_train, y_train)
pred_y = clf2.predict(X_test)

print(clf2.score(X_train, y_train))

# cause the linearSVR model is linear so the answer will out of range(1 to 5) 
# set the answer which is more than 5 to 5
for i in range(0,len(pred_y)):
    if(pred_y[i]>5):
        pred_y[i] = 5.0
    elif(pred_y[i]<1):
        pred_y[i] = 1.0
    else:
        pred_y[i] = round(pred_y[i],2)

ans2 = pred_y

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if __name__ == '__main__':


0.9087612666839255


In [4]:
# answer 0.80710.5042

# get the answer from linearSVC & linearSVR
# set the weighted 3:8
new = (ans1*3+ans2*8)/11

# to get the lower RMSE & higher accuracy
# reshape the range of the answer to 1.5~4.6 (not 1~5)
for i in range(0,len(new)):
    if(new[i]>=4.5):#4.4
        new[i] = 4.6
    elif(new[i]<1.5):
        new[i] = 1.5
    else:
        new[i] = round(new[i],3)

output = np.vstack((outid,new)).T
output = pd.DataFrame(output,columns=['user_id','stars'])
output['user_id'] = output['user_id'].astype(int)

# the answer is stored in the answer.csv
np.savetxt("answer.csv", output, fmt='%i, %1.4f', delimiter=",")

In [5]:
print(output.head(5))

   user_id  stars
0     2713  3.269
1     4734  4.600
2     5598  4.600
3     9545  4.600
4     1471  2.796
